# MINE 4201 - Laboratorio 2: Filtrado por Contenido y Estrategias de Embedding

En este laboratorio exploraremos los sistemas de recomendación basados en contenido, utilizando diferentes estrategias de representación vectorial (embeddings) para describir los ítems:

1. **TF-IDF** (Term Frequency – Inverse Document Frequency)
2. **Word2Vec** (Representaciones densas aprendidas)

Al finalizar, el estudiante será capaz de:
- Construir una matriz documento-término y calcular TF-IDF
- Aplicar selección de características para mejorar la representación
- Entrenar y evaluar un modelo de clasificación para recomendación por contenido
- Comprender el modelo Word2Vec y sus arquitecturas (Skip-gram, CBOW)
- Comparar las representaciones TF-IDF vs Word2Vec para filtrado por contenido
- Reflexionar sobre cómo estas estrategias de embedding se conectan con el Filtrado Colaborativo

---
## Contexto: Filtrado por Contenido vs Filtrado Colaborativo

En los sistemas de recomendación existen dos grandes familias de enfoques:

| Enfoque | Descripción | Datos que utiliza |
|---|---|---|
| **Filtrado por Contenido** | Recomienda ítems similares a los que el usuario ha preferido, basándose en las **características del ítem** (género, descripción, conceptos). | Atributos / features de los ítems |
| **Filtrado Colaborativo** | Recomienda ítems que usuarios similares han preferido, sin necesitar características del ítem. | Matriz de interacciones usuario-ítem |

### ¿Dónde entran los Embeddings?

Las estrategias de embedding (TF-IDF, Word2Vec, etc.) pueden utilizarse en **ambos** enfoques:

- **En filtrado por contenido:** se usan para representar las características de los ítems en un espacio vectorial continuo. Es lo que haremos en la primera parte de este laboratorio.
- **En filtrado colaborativo:** los embeddings pueden representar usuarios e ítems en un espacio latente compartido (e.g., factorización de matrices, embeddings neurales). Modelos como Word2Vec han inspirado técnicas como **Item2Vec**, donde se trata la secuencia de ítems consumidos por un usuario como una "oración" y se aprenden embeddings de ítems.

En este laboratorio nos centraremos en el **filtrado por contenido** usando TF-IDF y exploraremos Word2Vec como estrategia alternativa de representación.


Los sistemas de recomendación basados en contenido filtran contenido basado en la representación de items y el perfil del usuario. En este laboratorio trabajaremos con un conjunto de datos del sitio web [LibraryThing](https://https://www.librarything.com/).


## Preparación del entorno
Instale las librerias que vamos a utilizar e importelas en el ambiente de ejecución

In [ ]:
!pip install scikit-learn
!pip install pandas
!pip install seaborn

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support



## Carga de archivos

Copie el archivo del dataset en el entorno colab en la pestaña files (la carpeta en el menú de la izquierda), carguemos el csv en un dataframe de pandas y revisemos su contenido.

In [ ]:
if not os.path.exists('DB-BOOK-content.csv') :
  raise ValueError('El archivo DB-BOOK-content.csv no fue encontrado en el path')
else:
  print("Los archivos han sido cargados")

In [ ]:
df_dbbook=pd.read_csv('DB-BOOK-content.csv', sep=';')
df_dbbook

Este dataset tiene un formato similar al que manejamos el taller pasado. Tiene una columna con el id del usuario, otra con el id del item y un rating.

Cada rating esta presente varias veces, una por cada característica de los libros, revisemos por ejemplo las características del libro con ID 8010.

Se selecciona del dataframe las columnas name, featureID y featureShortname, la instruccion drop_duplicates nos deja solamente las columnas que no

Nota: Para saber más de indexación de dataframes de pandas utilizando .loc hay una explicación en la [documentación](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)



In [ ]:
df_dbbook.loc[df_dbbook.DBbook_ItemID==8010,['DBbook_ItemID','name','featureID','featureShortname']].drop_duplicates()

Cada libro tiene features que fueron extraidos de [DBpedia](https://wiki.dbpedia.org/). Dbpedia es una iniciativa para construir una representación de conceptos y relaciones mediante ontologías utilizando la información depositada en wikipedia. Más adelante vamos a utilizar DBPedia para otros talleres que aprovechan la información de la ontología de DBPedia, por ahora lo que tenemos es una representación de conjunto de palabras (o conceptos) para describir un item.  

## Creación de matriz documento-termino

En las siguientes líneas vamos a crear la matriz documento término, el primer paso es obtener en un dataframe los libros, los conceptos, y  los conceptos únicos por libro.

In [ ]:
df_libros=df_dbbook.loc[:,['DBbook_ItemID','name']].drop_duplicates()
df_libros

In [ ]:
df_conceptos=df_dbbook.loc[:,['featureID','featureShortname']].drop_duplicates()
df_conceptos

In [ ]:
df_libros_concepto=df_dbbook.loc[:,['DBbook_ItemID','name','featureID','featureShortname']].drop_duplicates()
df_libros_concepto

Se quiere obtener una martiz que tenga como filas cada uno de los libros, y como columnas cada uno de los conceptos, y en cada coordenada un 1 si esta presente el concepto y 0 de lo contrario. A esta operación se le conoce como pivot de una tabla.

La función [pivot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html) toma tres parámetros: la columna del dataframe original que va a tomar el índice de las filas del nuevo dataframe (index), la columna del dataframe original mediante la cual se van a generar las columnas (columns) y values los valores con los cuales se va a llenar el dataframe, en este caso  vamos a dejar momentaneamente el id del feature para indicar que existe.

Finalmente aplicamos la función [notna](https://pandas.pydata.org/docs/reference/api/pandas.notna.html) para modificar uno a uno los elementos de la matriz



In [ ]:

df_matriz_libros_concepto = (
    df_libros_concepto[['DBbook_ItemID', 'featureID']]
      .drop_duplicates()
      .pivot(index='DBbook_ItemID', columns='featureID', values='featureID')
      .notna()              # vectorized (fast)
      .astype('int8')       # 1/0 in int8
)

In [ ]:
df_matriz_libros_concepto

In [ ]:
df_matriz_libros_concepto.shape

En la siguiente celda se está aplicando la función [sumatoria](https://www.geeksforgeeks.org/python-pandas-dataframe-sum/) por eje al dataframe anterior, por defecto el eje es 0, por lo que generará una Serie (vector) del tamaño de las columnas y por cada columna calculará la sumatoria

In [ ]:
help(pd.DataFrame.sum)

In [ ]:
series_suma=df_matriz_libros_concepto.sum()
series_suma

El objeto series_suma no es un dataframe sino un objeto tipo Series, que es un arreglo. Un dataframe puede ser visto como una concatenación de varios objetos de tipo Series

In [ ]:
type(series_suma)

**Interprete las siguientes figuras y diga qué quieren decir en términos del número de items y de características asignadas a los items**

In [ ]:
sns.scatterplot(x=range(0,len(series_suma)) ,y=series_suma.sort_values() )

In [ ]:
series_suma_2=df_matriz_libros_concepto.sum(axis=1)
series_suma_2

In [ ]:
sns.scatterplot(x=range(0, len(series_suma_2)), y=series_suma_2.sort_values())

# Cálculo de matriz tf-idf

La matriz df_matriz_libros_concepto hasta el momento tiene en cada coordenada la presencia o ausencia de la característica que describe el atributo, este sería el término $\text{tf}$ de la siguiente fórmula donde $i$ es un término o palabra y $d$ es un documento.

$\text{tfidf}_{i,d} = \text{tf}_{i,d} \cdot \text{idf}_{i}$

El Inverse Document Frequency esta definido como:

$\text{idf}_{i,d} = \log \frac{N}{\text{df}_{i}}$

Donde $\text{df}_{t}$ es el número de documentos en los que aparece el término $i$ y N el número total de documentos



En python es posible operar vectores con escalares directamente, gracias a que por debajo python genera operaciones entre arreglos cuando operamos con un escalar mediante la operación de [broadcasting](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html) Por ejemplo, el anterior arreglo se puede dividir entre 6, por debajo python genera un arreglo del mismo tamaño y realiza la operación elemento a elemento

In [ ]:
series_suma/6

En las siguientes celdas, cree un objeto tipo series llamado df_idf, que contiene el IDF de cada atributo. Puede utilizar la función [np.log2](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log2.html) de numpy

In [ ]:
df_idf=???
df_idf

Si df_idf fue calculado correctamente, la siguiente instrucción multiplicará cada fila del dataframe elemento por elemento (element-wise) por la serie que contiene el IDF

In [ ]:
df_matriz_tf_idf=df_matriz_libros_concepto.multiply(df_idf, axis=1)
df_matriz_tf_idf

En el siguiente mapa de calor se observa el tf_idf de los items representados en las 300 características con mayor frecuencia.

In [ ]:
df_matriz_tf_idf.loc[:,series_suma.nlargest(300).index]

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
sns.heatmap(df_matriz_tf_idf.loc[:,series_suma.nlargest(300).index],cmap="Blues", vmin=0)

**¿Qué puede interpretar sobre la figura anterior?
Encuentre los nombres de las características más frecuentes**

# Selección de características

Una vez realizado el proceso de indexamiento, se puede realizar el proceso de selección de características.

En este momento contamos con más de 12 mil conceptos. ¿Con cuántos vale la pena crear los modelos de filtrado?

El paso más simple es filtrar las características con baja frecuencia dentro del dataset

In [ ]:
series_suma.describe()

**¿Qué puede decir sobre la distribución de frecuencia de las características en los items? ¿Vale la pena tener todas las características que tenemos actualmente?**

.
**Retire de la matriz df_matrix_tf_idf las columnas que representan a los items que tienen menos de 3 items asociados**

In [ ]:
# Se filtra la serie por aquellos que tienen al menos 3
series_suma[series_suma>=3]

In [ ]:
df_matriz_tf_idf=???

In [ ]:
df_matriz_tf_idf

### Para las siguientes estrategias de selección de características, tenemos que aplicar técnicas supervisadas (que conocen la clase a predecir o lo que se quiere pronosticar), para esto tenemos que retomar nuestro dataset original de interacciones entre usuarios e items para asignar la etiqueta (le gustó/ no le gustó)

In [ ]:
# Recordemos como es el dataset original.
df_dbbook.head(20)

Creemos un dataframe para crear un dataset de un sistema de recomendación como el visto en el laboratorio pasado (una única interacción de tipo usuario, item y rating)

In [ ]:
df_all_interactions=df_dbbook[['DBbook_userID','DBbook_ItemID','rate']].drop_duplicates()
df_all_interactions

Para asignar una clase, se binarizan los ratings. Una regla simple es calcular el rating promedio por persona. Todo lo que esté por debajo del promedio se clasifica como no le gusta, igual o por encima es si le gusta.

In [ ]:
df_user_mean=df_all_interactions.groupby('DBbook_userID')['rate'].mean().reset_index()
df_user_mean.columns=['DBbook_userID','mean']
df_user_mean

**Realice un merge entre df_all_interactions y df_user_mean, asignando su resultado a df_all_interactions.
Cree una nueva columna en el dataframe df_all_interactions llamada 'class' con True si el rating del usuario es mayor o igual a su promedio**

[Documentación pandas merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

[Tutorial creación de columnas a partir del valor de otras](https://thispointer.com/python-pandas-how-to-add-new-columns-in-a-dataframe-using-or-dataframe-assign/)

In [ ]:
df_all_interactions=???

In [ ]:
df_all_interactions['class']=???

In [ ]:
df_all_interactions

Por último, se procede a partir el dataset en entrenamiento y test. Se utiliza de la librería sklearn la función [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

Esta recibe como parámetro el dataset a partir, el porcentaje para test y un parámetro que indica la variable por la cual estratificar la partición, en este caso quisieramos que las interacciones en test sean de usuarios que vimos en train, por lo tanto se deja estratificado por usuario.

In [ ]:
help(train_test_split)

In [ ]:
#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)
df_all_interactions_train, df_all_interactions_test =train_test_split(df_all_interactions, test_size=0.2, stratify=df_all_interactions['DBbook_userID'])

In [ ]:
df_all_interactions_train

In [ ]:
df_all_interactions_test

In [ ]:
df_all_interactions_train.DBbook_userID.value_counts()

In [ ]:
df_all_interactions_test.DBbook_userID.value_counts()

In [ ]:
df_conteos_usuario_train_test=pd.concat([df_all_interactions_train.DBbook_userID.value_counts(),df_all_interactions_test.DBbook_userID.value_counts()],axis=1)
df_conteos_usuario_train_test.columns=['train_count','test_count']

In [ ]:
df_conteos_usuario_train_test

In [ ]:
df_conteos_usuario_train_test.nlargest(500,'test_count')

**Chi-cuadrado**

La selección de características mediante la prueba [chi-cuadrado](https://en.wikipedia.org/wiki/Chi-squared_test) nos dice si la diferencia observada entre las frecuencias de co-ocurrencia de dos variables es significativa. La idea es seleccionar características que más ayuden a discriminar la clase objetivo observando la frecuencia en la que ocurren juntas.

La librería sklearn permite identificar la importancia de cada una de las variables utilizando el método [chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html)

In [ ]:
help(chi2)

En filtrado por contenido, se arma un modelo por usuario. Por ahora vamos a escoger las características más importantes para el primer usuario del dataset de test

In [ ]:
#Debería dar 2817
primer_usuario_id=df_all_interactions_test.iloc[0]['DBbook_userID']
primer_usuario_id

Obtengamos los ids de los items con los que ha interactuado y su opinion

In [ ]:
df_temporal_usuario=df_all_interactions_train.loc[df_all_interactions_train.DBbook_userID==1124,['DBbook_ItemID','class']]
df_temporal_usuario

Peguemos a este dataframe la representación vectorial de tf_idf por el id del item, en la matriz es el índice de las filas

In [ ]:
df_temporal_usuario=df_temporal_usuario.merge(df_matriz_tf_idf, how='left', left_on='DBbook_ItemID', right_index=True)
df_temporal_usuario

Este dataframe representa los datos de entrenamiento del modelo para predicción de una clase binaria (class True es le gusta, class False es no le gusta)

In [ ]:
#vamos a indexar solamente las columnas que son características, la prueba chi2 lo compara todas las características contra la clase objetivo
features=df_matriz_tf_idf.columns

In [ ]:
pesos_chi2, pval= chi2(df_temporal_usuario[features],df_temporal_usuario['class'])

In [ ]:
#La prueba puede arrojar nan
pesos_chi2

In [ ]:
pval

In [ ]:
# LLenamos con peso 0 los que no se pudieron calcular
pesos_chi2=np.nan_to_num(pesos_chi2)
pesos_chi2

Se crea una máscara de indexación con los valores que son positivos según la prueba

In [ ]:
pesos_chi2_mask=pesos_chi2>0

In [ ]:
pesos_chi2_mask

In [ ]:
features[pesos_chi2_mask]

El siguiente sería el resultado, se recortaron las columnas de pesos del usuario, dejando 119 features.

In [ ]:
df_temporal_usuario.loc[:,features[pesos_chi2_mask]]

In [ ]:
del df_temporal_usuario

**Complete el código de la siguiente celda, el objetivo es crear un diccionario donde la llave es el id del usuario y el valor es un arreglo con los features seleccionados para el usuario**

El proceso que implementamos no esta optimizado, por lo que vamos a armar el modelo solamente para los 500 usuarios con más ratings en el dataset de test.


In [ ]:
diccionario_usuarios_features={}
# 500 usuarios con más ratings en test
unique_users_test=df_conteos_usuario_train_test.nlargest(500,'test_count').index
i=0
print(unique_users_test.shape)
for user in unique_users_test:
  if not user in diccionario_usuarios_features:
    df_temporal_usuario=df_all_interactions_train.loc[df_all_interactions_train.DBbook_userID==user,['DBbook_ItemID','class']]

    df_temporal_usuario=df_temporal_usuario.merge ????

    pesos_chi2, pval=????
    pesos_chi2=np.nan_to_num(pesos_chi2)
    pesos_chi2_mask=pesos_chi2>0
    features_usuario=features[pesos_chi2_mask]
    diccionario_usuarios_features[user]=features_usuario
    i=i+1
    if i%50==0:
      print(i)
    del df_temporal_usuario






In [ ]:
len(diccionario_usuarios_features)

In [ ]:
diccionario_usuarios_features.keys()

Otro criterio que puede ser usado es [Mutual Information](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

# Modelo de recomendación y evaluación

Una vez seleccionadas las características por usuario, se puede usar el dataset de entrenamiento para aprender un modelo de clasificación binaria y probarlo sobre test.

Uno de los modelos que puede ser utilizado es el [clasificador por vecinos más cercanos](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm).

Cada usuario tiene unas instancias representadas en un espacio vectorial del tamaño de las características seleccionadas anteriormente. Para una nueva instancia (predicción) se mira cuáles son los k vecinos más cercanos a ese dato nuevo y se predice la clase mayoritaria dentro del grupo de los vecinos. Observe el siguiente ejemplo.

In [ ]:
%%html
<iframe src="https://es.wikipedia.org/wiki/K_vecinos_m%C3%A1s_pr%C3%B3ximos#/media/Archivo:KnnClassification.svg" width="1200" height="600"></iframe>

Tomemos como ejemplo el usuario 3852, armemos su conjunto de entrenamiento. Note que se estan seleccionando solamente los features calculados en el punto anterior

In [ ]:
usuario_id = 3852
if usuario_id not in diccionario_usuarios_features:
    usuario_id = next(iter(diccionario_usuarios_features))
    print(f"Usuario 3852 no disponible en entrenamiento. Se usa usuario {usuario_id}.")

features_usuario = diccionario_usuarios_features[usuario_id]
df_temporal_usuario_train = df_all_interactions_train.loc[
    df_all_interactions_train.DBbook_userID == usuario_id, ['DBbook_ItemID', 'class']
 ]
df_temporal_usuario_train = df_temporal_usuario_train.merge(
    df_matriz_tf_idf[features_usuario], how='left', left_on='DBbook_ItemID', right_index=True
)
df_temporal_usuario_train

Armamos de igual forma el conjunto de test

In [ ]:
df_temporal_usuario_test = df_all_interactions_test.loc[
    df_all_interactions_test.DBbook_userID == usuario_id, ['DBbook_ItemID', 'class']
 ]
df_temporal_usuario_test = df_temporal_usuario_test.merge(
    df_matriz_tf_idf[features_usuario], how='left', left_on='DBbook_ItemID', right_index=True
)
df_temporal_usuario_test

Utilizaremos la clase [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) de scikit-learn para hacer la predicción de los datos del conjunto de test. El modelo tiene 3 métodos principales. El constructor permite inicializar el k a usar, la métrica entre otros; fit sirve para darle los datos de entrenamiento base al modelo; y predict para predecir los datos que se le pasan.

In [ ]:
#Con esta configuración se utilizan los 3 vecinos más cercanos, con distancia euclidiana
knn_clasif=KNeighborsClassifier(3)

In [ ]:
# Fit recibe la matriz de entrenamiento y la clase objetivo
knn_clasif.fit(df_temporal_usuario_train[features_usuario], df_temporal_usuario_train['class'])

In [ ]:
# llamamos predict sobre  los test , creando una nueva columna en el dataframe de test
df_temporal_usuario_test['predict']=knn_clasif.predict(df_temporal_usuario_test[features_usuario])

In [ ]:
df_temporal_usuario_test[['DBbook_ItemID','class','predict']].merge(df_libros, how='left', on='DBbook_ItemID')

**En las siguientes celdas realice hipótesis sobre por qué falló la clasificación para estos items y por qué funcionó para los otros, revise los conceptos seleccionados para el usuario y los asociados a los items**

Finalmente, la librería sklearn tiene diferentes métricas de evaluación de clasificación. En particular podemos calcular la matriz de confusión de la clasificación utilizando la función [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html), y el cálculo de las métricas precision, recall, y f1 con la función [precision_recall_fscore_support](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)

In [ ]:
tn, fp, fn, tp = confusion_matrix(df_temporal_usuario_test['class'],df_temporal_usuario_test['predict'], labels=[False,True]).ravel()

(tn, fp, fn, tp)

In [ ]:
precision_recall_fscore_support(df_temporal_usuario_test['class'],df_temporal_usuario_test['predict'], pos_label=True,average='binary')

**Ejercicio: Realice las predicciones binarias para los usuarios en el conjunto de test a los que se les hizo la selección de características, mida la precisión, el recall y f_score de su modelo con las predicciones realizadas y ajústelo cambiando el k**

---
# Parte 2: De TF-IDF a Embeddings Densos — Introducción a Word2Vec

Hasta aquí hemos trabajado con representaciones **dispersas** (sparse) basadas en TF-IDF. Estas tienen algunas limitaciones:

- **Alta dimensionalidad:** la matriz tiene miles de columnas (una por concepto)
- **Dispersión:** la mayoría de los valores son cero
- **Sin semántica:** dos conceptos sinónimos tienen columnas diferentes y no hay relación entre ellos

Las representaciones **densas** (como Word2Vec) resuelven estos problemas al aprender vectores de baja dimensionalidad donde conceptos similares están cerca en el espacio vectorial.

A continuación exploraremos paso a paso cómo funciona Word2Vec.

### Ejercicio de reflexión: TF-IDF vs Embeddings Densos

Antes de continuar con Word2Vec, reflexione sobre las siguientes preguntas:

1. ¿Qué ventajas y desventajas tiene la representación TF-IDF que utilizamos para los libros?
2. ¿Qué pasaría si dos libros tratan del mismo tema pero usan conceptos diferentes en DBpedia?
3. ¿Cómo cree que un embedding denso podría mejorar las recomendaciones?

In [ ]:
# Escriba sus respuestas como comentarios o en celdas markdown adicionales
# Respuesta 1:

# Respuesta 2:

# Respuesta 3:


## 1. Configuración Inicial

Primero importamos las librerías necesarias y configuramos el logging para ver el progreso del entrenamiento.

In [ ]:
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 2. ¿Por Qué Necesitamos Representar Palabras como Vectores?

Las computadoras no entienden texto directamente — necesitan números. La pregunta clave es: **¿cómo convertimos palabras en números de forma que se preserve su significado?**

### 2.1 El Modelo Bolsa de Palabras (Bag-of-Words)

El enfoque más simple es el modelo **Bolsa de Palabras** (*Bag-of-Words*). Este transforma cada documento en un vector de longitud fija donde cada elemento cuenta cuántas veces aparece una palabra.

**Ejemplo:** Dadas las oraciones:
- *"Juan quiere ver películas. María quiere películas también."*
- *"Juan también quiere ver fútbol. María odia el fútbol."*

El modelo genera vectores como:
- `[1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]`
- `[1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]`

Donde cada posición corresponde a una palabra del vocabulario.

### 2.2 Limitaciones de la Bolsa de Palabras

Este enfoque tiene **dos problemas fundamentales**:

1. **Pierde el orden de las palabras:** "Juan quiere a María" y "María quiere a Juan" producen vectores idénticos, aunque significan cosas distintas.

2. **No captura el significado:** Las palabras "bueno" y "excelente" podrían estar tan lejos en el espacio vectorial como "bueno" y "zapato", a pesar de que las dos primeras son sinónimos.

**Word2Vec resuelve el segundo problema:** aprende representaciones donde palabras con significados similares están cerca en el espacio vectorial.

## 3. ¿Cómo Funciona Word2Vec?

Word2Vec es un algoritmo basado en redes neuronales superficiales (*shallow neural networks*) que aprende representaciones vectoriales de palabras a partir de grandes cantidades de texto.

### 3.1 La Idea Central

> **"Conocerás una palabra por la compañía que mantiene"** — J.R. Firth, 1957

Word2Vec se basa en la **hipótesis distribucional**: palabras que aparecen en contextos similares tienden a tener significados similares. Por ejemplo, "perro" y "gato" aparecen frecuentemente con palabras como "mascota", "veterinario", "comida", etc.

### 3.2 Las Dos Arquitecturas

Word2Vec tiene dos variantes:

#### Skip-gram (SG)
- **Entrada:** una palabra central
- **Salida:** predice las palabras del contexto (vecinas)
- Ejemplo: dada la palabra "gato", predice "el", "come", "pescado"
- Funciona mejor con corpus pequeños y palabras poco frecuentes

#### Continuous Bag-of-Words (CBOW)
- **Entrada:** las palabras del contexto
- **Salida:** predice la palabra central
- Ejemplo: dadas "el", "come", "pescado", predice "gato"
- Es más rápido de entrenar y funciona bien con palabras frecuentes

### 3.3 ¿Qué aprende la red?

La red neuronal tiene una capa oculta. Los **pesos de la capa de proyección** (entre la entrada y la capa oculta) son los **vectores de las palabras** (embeddings). Si la capa oculta tiene 300 neuronas, obtendremos embeddings de 300 dimensiones.

El resultado son vectores con propiedades algebraicas notables, como:

- `vec("rey") - vec("hombre") + vec("mujer") ≈ vec("reina")`
- `vec("París") - vec("Francia") + vec("Japón") ≈ vec("Tokio")`

## 4. Demostración con un Modelo Pre-entrenado

Antes de entrenar nuestro propio modelo, veamos qué puede hacer Word2Vec usando un modelo ya entrenado con parte del dataset de Google News (~3 millones de palabras y frases).

> **Nota:** El modelo pesa aproximadamente 2GB. Si no tienes buena conexión, puedes saltar a la sección 5 (Entrenar tu propio modelo).

In [ ]:
%pip install gensim
%pip install plotly

In [ ]:
import gensim.downloader as api

# Descargamos el modelo pre-entrenado de Google News (300 dimensiones)
wv = api.load('word2vec-google-news-300')

### 4.1 Explorar el Vocabulario

Podemos ver las primeras palabras del vocabulario del modelo:

In [ ]:
# Mostramos las primeras 10 palabras del vocabulario
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"Palabra #{index}/{len(wv.index_to_key)}: {word}")

### 4.2 Obtener el Vector de una Palabra

Cada palabra está representada por un vector de 300 dimensiones:

In [ ]:
# Obtenemos el vector de la palabra 'king'
vec_king = wv['king']
print(f"Dimensiones del vector: {vec_king.shape}")
print(f"Primeros 10 valores: {vec_king[:10]}")

### 4.3 Palabras Desconocidas

Una limitación de Word2Vec es que **no puede generar vectores para palabras que no están en su vocabulario**. Si necesitas manejar palabras desconocidas, considera usar **FastText**, que trabaja con sub-palabras.

In [ ]:
# Intentamos obtener el vector de una palabra que no existe en el modelo
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("La palabra 'cameroon' no existe en este modelo")

### 4.4 Similitud entre Palabras

Word2Vec nos permite calcular la **similitud coseno** entre pares de palabras. Observa cómo la similitud disminuye intuitivamente a medida que las palabras son menos relacionadas:

In [ ]:
# Comparamos la similitud entre diferentes pares de palabras
pares = [
    ('car', 'minivan'),    # una minivan es un tipo de auto
    ('car', 'bicycle'),    # aún es un vehículo con ruedas
    ('car', 'airplane'),   # un vehículo, pero sin ruedas
    ('car', 'cereal'),     # sin relación aparente
    ('car', 'communism'),  # conceptos totalmente diferentes
]
for w1, w2 in pares:
    print(f'{w1:15s} {w2:15s} similitud: {wv.similarity(w1, w2):.4f}')

### 4.5 Palabras Más Similares

Podemos encontrar las palabras más cercanas a un concepto dado:

In [ ]:
# Las 5 palabras más similares a 'car' y 'minivan'
print("Palabras más similares a 'car' + 'minivan':")
for palabra, similitud in wv.most_similar(positive=['car', 'minivan'], topn=5):
    print(f"  {palabra}: {similitud:.4f}")

### 4.6 Detección de Intrusos

Word2Vec puede identificar qué palabra **no pertenece** a un grupo:

In [ ]:
# ¿Cuál de estas palabras no encaja con las demás?
intruso = wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car'])
print(f"La palabra que no pertenece al grupo es: '{intruso}'")

## 5. Entrenar Tu Propio Modelo

Ahora viene la parte más importante: **entrenar un modelo Word2Vec con tus propios datos**.

### 5.1 Preparar los Datos

Word2Vec necesita como entrada un iterable de oraciones, donde cada oración es una lista de palabras (tokens). Usaremos el **Lee Evaluation Corpus** incluido en Gensim.

Implementamos un iterador que lee el corpus línea por línea, lo cual es eficiente en memoria para corpus grandes:

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MiCorpus:
    """Iterador que produce oraciones (listas de palabras)."""

    def __iter__(self):
        ruta_corpus = datapath('lee_background.cor')
        for linea in open(ruta_corpus):
            # Asumimos un documento por línea, tokens separados por espacios
            yield utils.simple_preprocess(linea)

### 5.2 Entrenar el Modelo

Entrenar un modelo Word2Vec con Gensim es sorprendentemente simple. Todo el preprocesamiento personalizado (minúsculas, eliminación de números, etc.) se puede hacer dentro del iterador — Word2Vec solo necesita que la entrada produzca listas de palabras.

In [ ]:
import gensim.models

# Creamos el iterador del corpus
oraciones = MiCorpus()

# Entrenamos el modelo Word2Vec
modelo = gensim.models.Word2Vec(sentences=oraciones)
print("¡Modelo entrenado exitosamente!")

### 5.3 Usar el Modelo Entrenado

Una vez entrenado, podemos usar nuestro modelo de la misma forma que el modelo pre-entrenado. Los vectores de palabras están en `modelo.wv` ("wv" = *word vectors*).

In [ ]:
# Obtener el vector de una palabra
vec_king = modelo.wv['king']
print(f"Vector de 'king' (primeros 10 valores): {vec_king[:10]}")

# Ver las primeras 10 palabras del vocabulario
print("\nPrimeras 10 palabras del vocabulario:")
for index, word in enumerate(modelo.wv.index_to_key):
    if index == 10:
        break
    print(f"  #{index}: {word}")

## 6. Guardar y Cargar Modelos

Entrenar modelos puede tomar tiempo, así que es importante poder **guardarlos en disco** y reutilizarlos después sin volver a entrenar.

In [ ]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='modelo-gensim-', delete=False) as tmp:
    ruta_temporal = tmp.name
    # Guardar el modelo en disco
    modelo.save(ruta_temporal)
    print(f"Modelo guardado en: {ruta_temporal}")

    # Cargar el modelo desde disco
    modelo_cargado = gensim.models.Word2Vec.load(ruta_temporal)
    print("Modelo cargado exitosamente")

También es posible cargar modelos creados con la herramienta original en C:

```python
# Formato texto
modelo = gensim.models.KeyedVectors.load_word2vec_format('/ruta/vectores.txt', binary=False)

# Formato binario (también acepta archivos comprimidos .gz o .bz2)
modelo = gensim.models.KeyedVectors.load_word2vec_format('/ruta/vectores.bin.gz', binary=True)
```

## 7. Parámetros de Entrenamiento

Word2Vec acepta varios parámetros que afectan tanto la velocidad como la calidad del entrenamiento. Entender estos parámetros es clave para obtener buenos resultados.

### 7.1 `min_count` — Frecuencia Mínima

Controla el filtrado del vocabulario. Las palabras que aparecen menos de `min_count` veces se ignoran. Esto elimina errores tipográficos y palabras demasiado raras para las cuales no hay suficientes datos de entrenamiento.

- **Valor por defecto:** 5

In [ ]:
# Solo considerar palabras que aparecen al menos 10 veces
modelo_mc = gensim.models.Word2Vec(oraciones, min_count=10)
print(f"Tamaño del vocabulario con min_count=10: {len(modelo_mc.wv)}")

### 7.2 `vector_size` — Dimensión de los Vectores

Define el número de dimensiones del espacio vectorial. Vectores más grandes pueden capturar relaciones más complejas, pero requieren más datos de entrenamiento y más memoria.

- **Valor por defecto:** 100
- **Valores típicos:** entre 50 y 300

In [ ]:
# Usar vectores de 200 dimensiones
modelo_vs = gensim.models.Word2Vec(oraciones, vector_size=200)
print(f"Dimensión de los vectores: {modelo_vs.wv.vector_size}")

### 7.3 `sg` — Seleccionar la Arquitectura

Este parámetro selecciona entre las dos arquitecturas de Word2Vec:
- `sg=0` → **CBOW** (valor por defecto)
- `sg=1` → **Skip-gram**

In [ ]:
# Entrenar con Skip-gram
modelo_sg = gensim.models.Word2Vec(oraciones, sg=1)
print("Modelo entrenado con arquitectura Skip-gram")

### 7.4 `workers` — Paralelización

Controla el número de hilos de CPU utilizados durante el entrenamiento. Más hilos = entrenamiento más rápido (requiere Cython instalado).

- **Valor por defecto:** 3

In [ ]:
# Entrenar usando 4 hilos
modelo_w = gensim.models.Word2Vec(oraciones, workers=4)
print("Modelo entrenado con 4 hilos de CPU")

### 7.5 Uso de Memoria

La memoria requerida por Word2Vec depende de dos factores:

- **Tamaño del vocabulario** (controlado por `min_count`)
- **Dimensión de los vectores** (`vector_size`)

La fórmula aproximada es:

$$\text{Memoria} \approx \text{vocabulario} \times \text{vector\_size} \times 4 \text{ bytes} \times 3 \text{ matrices}$$

Por ejemplo, con 100,000 palabras y `vector_size=200`:

$$100{,}000 \times 200 \times 4 \times 3 = 240 \text{ MB}$$

## 8. Evaluación del Modelo

Word2Vec es un modelo **no supervisado**, por lo que no existe una métrica universal para evaluarlo. Sin embargo, hay dos enfoques comunes:

### 8.1 Analogías de Palabras

Google publicó un conjunto de pruebas con ~20,000 analogías sintácticas y semánticas del tipo *"A es a B como C es a D"*:

- **Sintácticas:** `malo:peor :: bueno:?` → mejor
- **Semánticas:** `París:Francia :: Tokio:?` → Japón

In [ ]:
# Evaluar el modelo con analogías de palabras
resultados_analogias = modelo.wv.evaluate_word_analogies(datapath('questions-words.txt'))
print(f"Precisión en analogías: {resultados_analogias[0]:.4f}")

### 8.2 Similitud entre Pares de Palabras

Otro enfoque utiliza el dataset **WS-353**, que contiene pares de palabras con puntuaciones de similitud asignadas por humanos. Medimos qué tan bien las similitudes del modelo correlacionan con los juicios humanos.

In [ ]:
# Evaluar con el dataset de similitud WS-353
resultados_pares = modelo.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
print(f"Correlación de Pearson: {resultados_pares[0][0]:.4f}")
print(f"Correlación de Spearman: {resultados_pares[1][0]:.4f}")

> **Importante:** Un buen desempeño en estos benchmarks no garantiza que el modelo funcione bien en tu tarea específica. Siempre es mejor evaluar directamente en tu aplicación final.

## 9. Entrenamiento Incremental

Es posible cargar un modelo existente y **continuar entrenándolo** con nuevas oraciones y vocabulario adicional:

In [ ]:
# Cargar el modelo previamente guardado
modelo = gensim.models.Word2Vec.load(ruta_temporal)

# Nuevas oraciones para continuar el entrenamiento
nuevas_oraciones = [
    ['los', 'usuarios', 'avanzados', 'pueden', 'cargar', 'un', 'modelo',
     'y', 'continuar', 'entrenando', 'con', 'mas', 'oraciones'],
]

# Actualizar el vocabulario con nuevas palabras
modelo.build_vocab(nuevas_oraciones, update=True)

# Continuar el entrenamiento
modelo.train(nuevas_oraciones, total_examples=modelo.corpus_count, epochs=modelo.epochs)
print("Entrenamiento incremental completado")

# Limpiar el archivo temporal
import os
os.remove(ruta_temporal)

> **Nota:** No es posible continuar el entrenamiento de modelos cargados con `KeyedVectors.load_word2vec_format()`, ya que estos no contienen la información del árbol de vocabulario necesaria para el entrenamiento.

## 10. Cálculo de la Pérdida (Loss) durante el Entrenamiento

Podemos monitorear la pérdida durante el entrenamiento activando el parámetro `compute_loss`. Esto nos ayuda a verificar que el modelo está aprendiendo.

In [ ]:
# Entrenar con cálculo de pérdida activado
modelo_con_loss = gensim.models.Word2Vec(
    oraciones,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,   # Usamos Skip-gram
    seed=42,
)

# Obtener el valor de la pérdida del entrenamiento
perdida = modelo_con_loss.get_latest_training_loss()
print(f"Pérdida del entrenamiento: {perdida:.4f}")

## 11. Visualización de los Embeddings

Una forma intuitiva de entender lo que Word2Vec ha aprendido es **visualizar los vectores en 2D** usando la técnica t-SNE (*t-Distributed Stochastic Neighbor Embedding*).

En una buena visualización deberías poder observar:
- **Agrupaciones semánticas:** palabras como "perro", "gato", "vaca" aparecen juntas
- **Agrupaciones sintácticas:** palabras como "correr", "corriendo" están cerca
- **Relaciones vectoriales:** `vec(rey) - vec(hombre) ≈ vec(reina) - vec(mujer)`

> **Nota:** El modelo está entrenado con un corpus pequeño, por lo que las relaciones pueden no ser tan claras.

In [ ]:
from sklearn.decomposition import IncrementalPCA    # Reducción inicial
from sklearn.manifold import TSNE                   # Reducción final a 2D
import numpy as np


def reducir_dimensiones(modelo):
    """Reduce los embeddings a 2 dimensiones usando t-SNE."""
    num_dimensiones = 2

    # Extraer los vectores y las etiquetas como arrays de NumPy
    vectores = np.asarray(modelo.wv.vectors)
    etiquetas = np.asarray(modelo.wv.index_to_key)

    # Reducir dimensionalidad con t-SNE
    tsne = TSNE(n_components=num_dimensiones, random_state=0)
    vectores = tsne.fit_transform(vectores)

    x_vals = [v[0] for v in vectores]
    y_vals = [v[1] for v in vectores]
    return x_vals, y_vals, etiquetas


x_vals, y_vals, etiquetas = reducir_dimensiones(modelo)

In [ ]:
def graficar_con_plotly(x_vals, y_vals, etiquetas, en_notebook=True):
    """Visualiza los embeddings usando Plotly (interactivo)."""
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    traza = go.Scatter(x=x_vals, y=y_vals, mode='text', text=etiquetas)
    datos = [traza]

    if en_notebook:
        init_notebook_mode(connected=True)
        iplot(datos, filename='embedding-palabras')
    else:
        plot(datos, filename='embedding-palabras.html')


def graficar_con_matplotlib(x_vals, y_vals, etiquetas):
    """Visualiza los embeddings usando Matplotlib (estático)."""
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)
    plt.title('Visualización de Embeddings Word2Vec (t-SNE)')
    plt.xlabel('Dimensión 1')
    plt.ylabel('Dimensión 2')

    # Etiquetar 25 puntos seleccionados al azar
    indices = list(range(len(etiquetas)))
    indices_seleccionados = random.sample(indices, 25)
    for i in indices_seleccionados:
        plt.annotate(etiquetas[i], (x_vals[i], y_vals[i]))

    plt.show()


def _nbformat_disponible(min_major=4, min_minor=2):
    try:
        import nbformat
        partes = nbformat.__version__.split('.')
        major = int(partes[0])
        minor = int(partes[1]) if len(partes) > 1 else 0
        return (major, minor) >= (min_major, min_minor)
    except Exception:
        return False


# Seleccionar la función de graficación apropiada
try:
    get_ipython()
    en_notebook = True
except Exception:
    en_notebook = False

if en_notebook and _nbformat_disponible():
    funcion_graficar = graficar_con_plotly
else:
    if en_notebook:
        print('nbformat>=4.2.0 no está disponible; usando Matplotlib como alternativa.')
    funcion_graficar = graficar_con_matplotlib

funcion_graficar(x_vals, y_vals, etiquetas)

## 12. Resumen y Conceptos Clave

En este tutorial aprendimos:

| Concepto | Descripción |
|---|---|
| **Bolsa de Palabras** | Representación simple que pierde orden y significado |
| **Word2Vec** | Genera embeddings que capturan relaciones semánticas |
| **Skip-gram** | Predice el contexto a partir de una palabra central |
| **CBOW** | Predice la palabra central a partir del contexto |
| **Similitud coseno** | Mide qué tan parecidos son dos vectores |
| **t-SNE** | Técnica para visualizar vectores de alta dimensión en 2D |

### Parámetros más importantes:
- `vector_size`: Dimensión de los embeddings (más grande = más expresivo, pero necesita más datos)
- `min_count`: Frecuencia mínima para incluir una palabra
- `sg`: 0 para CBOW, 1 para Skip-gram
- `workers`: Número de hilos para paralelizar el entrenamiento

### Enlaces útiles:
- [Documentación de Gensim Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)
- [Artículos originales de Word2Vec por Google](https://code.google.com/archive/p/word2vec/)
- [Tutorial visual de Word2Vec (Jay Alammar)](https://jalammar.github.io/illustrated-word2vec/)

---
# Parte 3: Aplicación de Word2Vec al Dataset de Libros

Ahora que comprendemos cómo funciona Word2Vec, vamos a aplicar esta técnica al dataset de libros de LibraryThing que utilizamos en la Parte 1.

La idea es:
1. Tratar los conceptos (features) de cada libro como "palabras" y cada libro como una "oración"
2. Entrenar un modelo Word2Vec sobre estas "oraciones" para obtener embeddings de conceptos
3. Representar cada libro como el promedio de los embeddings de sus conceptos
4. Usar esta nueva representación para el modelo de recomendación y comparar con TF-IDF

### Paso 1: Preparar los datos como "oraciones" de conceptos

Cada libro tiene un conjunto de conceptos de DBpedia asociados. Vamos a crear una lista de "oraciones" donde cada oración es la lista de nombres cortos de conceptos (featureShortname) de un libro.

In [ ]:
# TODO: Crear una lista de oraciones a partir de los conceptos de cada libro
# Cada oración debe ser una lista de strings con los featureShortname de un libro
# Hint: Agrupe df_libros_concepto por DBbook_ItemID y recolecte los featureShortname

oraciones_libros = (
    df_libros_concepto
    .groupby('DBbook_ItemID')['featureShortname']
    .apply(list)
    .tolist()
)

print(f'Número de libros (oraciones): {len(oraciones_libros)}')
print(f'Ejemplo de oracion (primer libro): {oraciones_libros[0][:10]}...')

### Paso 2: Entrenar un modelo Word2Vec con los conceptos de los libros

**Complete la celda siguiente** entrenando un modelo Word2Vec con los parámetros que considere apropiados. Considere:
- `vector_size`: ¿cuántas dimensiones? (pruebe con 50 o 100)
- `window`: ¿qué tamaño de ventana? (los conceptos de un libro no tienen orden estricto)
- `min_count`: ¿frecuencia mínima?
- `sg`: ¿Skip-gram o CBOW?

In [ ]:
import gensim.models

# TODO: Entrene el modelo Word2Vec con las oraciones de conceptos de libros
# modelo_w2v = gensim.models.Word2Vec(
#     sentences=oraciones_libros,
#     vector_size=???,
#     window=???,
#     min_count=???,
#     sg=???,
#     workers=4
# )

modelo_w2v = ???

print(f'Vocabulario del modelo: {len(modelo_w2v.wv)} conceptos')
print(f'Dimensión de los embeddings: {modelo_w2v.wv.vector_size}')

### Paso 3: Explorar los embeddings aprendidos

Verifique que los embeddings tienen sentido buscando conceptos similares.

In [ ]:
# TODO: Explore los embeddings aprendidos
# Pruebe buscando los conceptos más similares a algún concepto del vocabulario
# Ejemplo: modelo_w2v.wv.most_similar('Novel', topn=10)



### Paso 4: Representar cada libro como el promedio de sus embeddings

Para obtener un vector por libro, calculamos el **promedio** de los embeddings de todos sus conceptos. Esto nos da una representación densa de cada libro.

In [ ]:
# TODO: Crear la matriz de representación de libros usando Word2Vec
# Para cada libro, calcule el promedio de los embeddings de sus conceptos

def obtener_embedding_libro(item_id, modelo_wv, df_libros_concepto):
    """Calcula el embedding promedio de un libro a partir de sus conceptos."""
    conceptos = df_libros_concepto.loc[
        df_libros_concepto.DBbook_ItemID == item_id, 'featureShortname'
    ].unique()
    vectores = []
    for c in conceptos:
        if c in modelo_wv.key_to_index:
            vectores.append(modelo_wv[c])
    if vectores:
        return np.mean(vectores, axis=0)
    else:
        return np.zeros(modelo_wv.vector_size)


# Construir la matriz de embeddings para todos los libros
ids_libros = df_libros['DBbook_ItemID'].values
embeddings_libros = np.array([
    obtener_embedding_libro(item_id, modelo_w2v.wv, df_libros_concepto)
    for item_id in ids_libros
])

df_matriz_w2v = pd.DataFrame(embeddings_libros, index=ids_libros)
print(f'Forma de la matriz Word2Vec: {df_matriz_w2v.shape}')
df_matriz_w2v.head()

### Paso 5: Entrenar el modelo KNN con representación Word2Vec

**Complete la celda siguiente** para entrenar un clasificador KNN usando la representación Word2Vec en lugar de TF-IDF, y evalúe los resultados para un usuario de ejemplo.

In [ ]:
# TODO: Repita el proceso de clasificación usando la representación Word2Vec
# 1. Seleccione un usuario del conjunto de test
# 2. Obtenga sus items de entrenamiento y test
# 3. Asocie la representación Word2Vec (df_matriz_w2v) en lugar de TF-IDF
# 4. Entrene un KNeighborsClassifier
# 5. Evalúe con confusion_matrix y precision_recall_fscore_support

# Ejemplo de estructura:
# usuario_ejemplo = ???
# df_train_w2v = df_all_interactions_train.loc[
#     df_all_interactions_train.DBbook_userID == usuario_ejemplo, ['DBbook_ItemID', 'class']
# ].merge(df_matriz_w2v, how='left', left_on='DBbook_ItemID', right_index=True)
#
# df_test_w2v = df_all_interactions_test.loc[
#     df_all_interactions_test.DBbook_userID == usuario_ejemplo, ['DBbook_ItemID', 'class']
# ].merge(df_matriz_w2v, how='left', left_on='DBbook_ItemID', right_index=True)
#
# features_w2v = df_matriz_w2v.columns
# knn_w2v = KNeighborsClassifier(3)
# knn_w2v.fit(df_train_w2v[features_w2v], df_train_w2v['class'])
# df_test_w2v['predict'] = knn_w2v.predict(df_test_w2v[features_w2v])
#
# tn, fp, fn, tp = confusion_matrix(df_test_w2v['class'], df_test_w2v['predict'], labels=[False, True]).ravel()
# print('Confusion matrix:', tn, fp, fn, tp)
# print(precision_recall_fscore_support(df_test_w2v['class'], df_test_w2v['predict'], pos_label=True, average='binary'))


---
## Comparación: TF-IDF vs Word2Vec para Filtrado por Contenido

Ahora que ha implementado ambos enfoques, compare los resultados.

### Ejercicio: Evaluación comparativa

1. Ejecute el modelo KNN con representación TF-IDF y Word2Vec para al menos 50 usuarios
2. Calcule el precision, recall y f-score promedio para cada representación
3. Compare los resultados y discuta cuál representación funciona mejor y por qué

In [ ]:
# TODO: Implemente la evaluación comparativa entre TF-IDF y Word2Vec
# Calcule las métricas promedio para ambos modelos sobre múltiples usuarios



### Ejercicio de reflexión: Comparación de estrategias

Responda las siguientes preguntas basándose en los resultados obtenidos:

1. ¿Cuál representación obtuvo mejores resultados? ¿Por qué cree que es así?
2. ¿Cuáles son las ventajas computacionales de cada enfoque? (dimensionalidad, tiempo de entrenamiento, memoria)
3. ¿En qué escenarios sería más apropiado usar TF-IDF vs Word2Vec?
4. ¿Cómo se podrían aplicar estas representaciones en un esquema de **Filtrado Colaborativo**? (Hint: piense en Item2Vec, donde las secuencias de ítems consumidos por un usuario se tratan como oraciones)

In [ ]:
# Escriba sus respuestas aquí como comentarios
# Respuesta 1:

# Respuesta 2:

# Respuesta 3:

# Respuesta 4:
